# Solving TSP with Dynamic Programming

### Import the required libraries

In [1]:
import sys
import copy
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
top_dests_ktm=pd.read_csv("../../Datasets/destinations_of_kathmandu_updated_with_latlong.csv")
print(top_dests_ktm.head())

                  title avg_rating  voted_by            genre    latitude  \
0      Boudhanath Stupa        4.5      8897  Religious Sites  27.7215062   
1  Swayambhunath Temple        4.5      6203  Religious Sites  27.7149298   
2  Pashupatinath Temple        4.5      4937  Religious Sites  27.7104461   
3     Chandragiri Hills        4.5       399        Mountains  27.6710496   
4       Kopan Monastery        4.5       787  Religious Sites  27.7425438   

   longitude  
0  85.359809  
1  85.288146  
2  85.346503  
3  85.262664  
4  85.362208  


In [3]:
# select the first 95 destinations as they're the destinations having the latitude and longitude currently
top_dests_ktm=top_dests_ktm[:95]
print(top_dests_ktm.tail())

                         title avg_rating  voted_by  \
90     Ghanta Ghar Clock Tower        3.5        11   
91  Jamchen Lhakhang Monastery        3.5         3   
92                    Big Bell        3.5         9   
93       Mrigasthali Deer Park          4         5   
94            Universal Crafts          5         3   

                              genre    latitude  longitude  
90   Points of Interest & Landmarks   27.707477  85.314711  
91                  Religious Sites  27.7215058  85.359192  
92   Points of Interest & Landmarks  27.7268583  85.300625  
93  Nature & Wildlife Areas • Parks   27.711512  85.349772  
94    Art Galleries • Antique Shops  27.7173331  85.381252  


In [4]:
# let's just pick n=10 for now for better visualization
n=25 #graph size
top_n_dests=top_dests_ktm[:n]
print(top_n_dests)

                               title avg_rating  voted_by  \
0                   Boudhanath Stupa        4.5      8897   
1               Swayambhunath Temple        4.5      6203   
2               Pashupatinath Temple        4.5      4937   
3                  Chandragiri Hills        4.5       399   
4                    Kopan Monastery        4.5       787   
5                             Thamel          4      5687   
6                   Garden of Dreams        4.5      4021   
7                Namo Buddha (Stupa)        4.5       129   
8            Kathmandu Durbar Square          4      4711   
9                               Asan        4.5       243   
10  Shivapuri Nagarjun National Park        4.5       135   
11               Kailashnath Mahadev        4.5       112   
12                Dakshinkali Temple        4.5       235   
13                    Budhanilkantha        4.5       372   
14                     Hanuman Dhoka        4.5       589   
15                    Th

In [5]:
# since we'll be dealing only with latitude and longitude at the moment
# only filter those columns along with the title
print(top_dests_ktm[['title','latitude','longitude']])

                         title    latitude  longitude
0             Boudhanath Stupa  27.7215062  85.359809
1         Swayambhunath Temple  27.7149298  85.288146
2         Pashupatinath Temple  27.7104461  85.346503
3            Chandragiri Hills  27.6710496  85.262664
4              Kopan Monastery  27.7425438  85.362208
..                         ...         ...        ...
90     Ghanta Ghar Clock Tower   27.707477  85.314711
91  Jamchen Lhakhang Monastery  27.7215058  85.359192
92                    Big Bell  27.7268583  85.300625
93       Mrigasthali Deer Park   27.711512  85.349772
94            Universal Crafts  27.7173331  85.381252

[95 rows x 3 columns]


In [6]:
# also meanwhile let's create a matrix called 'graph' to store the weights for each edges
graph=np.zeros((n,n))

def create_graph():

    # iterators to iterate through the graph
    i=0
    j=0

    for lat,long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
        j=0
        for another_lat,another_long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
            #edge weight(Euclidean distance)
            distance_between_the_places=np.sqrt((lat-another_lat)**2+(long-another_long)**2)
            #print(distance_between_the_places)

            #and store it in the 'graph' matrix
            graph[i][j]=distance_between_the_places
            #increment j 
            j+=1

        #increment i
        i+=1

In [7]:
create_graph()
print(graph)

[[0.00000000e+00 7.19642198e-02 1.73023925e-02 1.09466713e-01
  2.11739985e-02 5.23200347e-02 4.80632856e-02 2.65469904e-01
  5.79962199e-02 5.15515319e-02 9.08698321e-02 1.35237058e-01
  1.52500630e-01 8.29027080e-02 5.72366815e-02 2.03088614e-02
  9.55999053e-02 4.41563252e-02 4.78888725e-02 5.79515616e-02
  1.53031827e-01 5.53390577e-02 1.69994833e-02 1.25888449e-01
  6.95355546e-02]
 [7.19642198e-02 0.00000000e+00 5.85291924e-02 5.07423284e-02
  7.90430371e-02 1.97482572e-02 2.41831587e-02 3.23900887e-01
  1.95451193e-02 2.32776623e-02 1.25242198e-01 1.96384843e-01
  1.13060795e-01 5.28757482e-02 2.01347297e-02 5.87463917e-02
  2.45951652e-02 2.79942202e-02 3.83278269e-02 1.95640202e-02
  1.75116321e-01 5.42302618e-02 8.22021534e-02 1.48286968e-01
  2.82960385e-02]
 [1.73023925e-02 5.85291924e-02 0.00000000e+00 9.26338415e-02
  3.57340445e-02 3.88777444e-02 3.43848908e-02 2.70603313e-01
  4.25048371e-02 3.64227735e-02 1.03713982e-01 1.41096425e-01
  1.35495561e-01 7.97699581e-02 4.

In [8]:
# input matrix containing cost
matrix = graph
# input data points
data = list(range(1,n+1))

In [9]:
# initializing necessary parameters
all_sets = []
g = {}
p = []
starting_node=1

In [10]:
def main():
    """
        Main function
    """
    
    route=[] #to store the final route
    
    for x in range(1, n):
        g[x + 1, ()] = matrix[x][0]
    
    # minimum cost
    min_cost=get_minimum(starting_node, tuple(range(2,n+1)))
#     print("Miniumum cost: ",min_cost)
#     print('\n\nSolution to TSP: {1, ', end='')
    
    route.append(1) #append the starting node
    
    solution = p.pop()
#     print(solution[1][0], end=', ')
    route.append(solution[1][0])
    for x in range(n - 2):
        for new_solution in p:
            if tuple(solution[1]) == new_solution[0]:
                solution = new_solution
                print(solution[1][0], end=', ')
                route.append(solution[1][0])
                break
#     print('1}')
    route.append(1)#append the ending node (same as starting node)
#     print(route)
    return route

In [11]:
def get_minimum(k, a):
    """
        Get the minimum cost
    """
    if (k, a) in g:
        # Already calculated Set g[%d, (%s)]=%d' % (k, str(a), g[k, a]))
        return g[k, a]

    values = []
    all_min = []
    for j in a:
        set_a = copy.deepcopy(list(a))
        set_a.remove(j)
        all_min.append([j, tuple(set_a)])
        result = get_minimum(j, tuple(set_a))
        values.append(matrix[k-1][j-1] + result)

    # get minimun value from set as optimal solution for
    g[k, a] = min(values)
    p.append(((k, a), all_min[values.index(g[k, a])]))

    return g[k, a]


In [ ]:
if __name__ == '__main__':
    import time
    t_in=time.time()
    route=main()
    t_out=time.time()
    print(route)

In [ ]:
#cross-checking the total cost
total_cost=0 #initialize total cost to 0
for i in range(0,len(route)-1):
    total_cost=total_cost+matrix[route[i]-1][route[i+1]-1]
print(total_cost)

In [ ]:
for i in route:
    print(top_n_dests.iloc[i-1]["title"],end='')
    print("--->",end='')

In [ ]:
#actual distance given by Vincenty distance using more accurate ellipsoidal models such as WGS-84 than Haversine
import geopy.distance
total_distance=0 #total actual distance

for i in range(len(route)-1):
    coords_1 = (top_n_dests.iloc[route[i]-1]["latitude"],top_n_dests.iloc[route[i]-1]["longitude"])
    coords_2 = (top_n_dests.iloc[route[i+1]-1]["latitude"],top_n_dests.iloc[route[i+1]-1]["longitude"])
    
    #names of destinations connected
    src=top_n_dests.iloc[route[i]-1]["title"]
    dest=top_n_dests.iloc[route[i+1]-1]["title"]
    distance=geopy.distance.geodesic(coords_1, coords_2).km
    print (f'{str(src)+"->"+str(dest)}',distance)
    total_distance=total_distance+distance

print("Total distance(km): ",total_distance)
print(t_out-t_in)